In [7]:
'''
These two models have resized input images to 512x512
So, first, take the original height and width of image, then resize it to 512x512
Then, use the model to predict the mask
Find the contour of the mask and resize it back to the original height and width
Find the ellipse using  OpenCV cv2.fitEllipse() function
Get the major and minor axis of the ellipse
Calculate HC = 1.62 x (BPD + OFD)
where BPD is the major axis and OFD is the minor axis
find HC in mm using the formula HC_mm = HC x pixel_size
where pixel size is taken from CSV file
find MSE between the predicted HC and the actual HC
'''

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import load_model
from skimage.transform import resize
from skimage import measure
from sklearn.metrics import mean_squared_error
import os



In [23]:
import numpy as np
import cv2
from skimage.transform import resize
from skimage import measure
from sklearn.metrics import mean_squared_error

# Preprocess image: Normalize and resize
def preprocess_image(image):
    image = image / 255.0  # Normalize
    image = resize(image, (512, 512), mode='constant', preserve_range=True)  # Resize
    return image

# Postprocess mask: Resize back and find ellipse
def postprocess_mask(mask, image):
    mask = resize(mask, (image.shape[0], image.shape[1]), mode='constant', preserve_range=True)
    mask = mask > 0.5  # Convert to binary mask
    
    # Find contours
    contours = measure.find_contours(mask, 0.5)
    if len(contours) == 0:
        return None
    
    # Select the largest contour
    contour = max(contours, key=len)
    contour = np.array(contour, dtype=np.float32)  # Convert to float
    contour = contour.reshape(-1, 1, 2)  # Reshape for OpenCV

    # Fit ellipse
    if len(contour) < 5:  # At least 5 points needed
        return None
    ellipse = cv2.fitEllipse(contour)
    return ellipse

# Calculate Head Circumference (HC) from ellipse
def calculate_HC(ellipse, pixel_size):
    major_axis = ellipse[1][0]
    minor_axis = ellipse[1][1]

    # Corrected formula for ellipse circumference (Ramanujan's approximation)
    HC = np.pi * (3 * (major_axis + minor_axis) - np.sqrt((3 * major_axis + minor_axis) * (major_axis + 3 * minor_axis)))
    
    # Convert to mm
    HC_mm = HC * pixel_size
    return HC_mm

# Calculate Mean Squared Error (MSE)
def calculate_MSE(predicted_HC, actual_HC):
    return mean_squared_error([actual_HC], [predicted_HC])  # Ensure both are lists


In [11]:
df = pd.read_csv('Dataset/training_set_pixel_size_and_HC.csv')

In [28]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Define the Dice Loss function
def dice_loss(y_true, y_pred):
    smooth = 1.0
    intersection = tf.reduce_sum(y_true * y_pred)
    return 1 - (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

# Define the combined BCE + Dice loss
def combined_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    return bce + dice  # This ensures both losses are accounted for

u_net_model = 'Model Weights/fetal_head_segmentation_unet.h5'
res_u_net_model = 'Model Weights/fetal_head_segmentation_resunet.h5'

# Load the models with the custom loss
unet = load_model(u_net_model, custom_objects={'combined_loss': combined_loss, 'dice_loss': dice_loss})
resunet = load_model(res_u_net_model, custom_objects={'combined_loss': combined_loss, 'dice_loss': dice_loss})


In [33]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
from skimage.transform import resize
from skimage import measure
from sklearn.metrics import mean_squared_error

# Define Combined Loss Function (Binary Crossentropy + Dice Loss)
def dice_loss(y_true, y_pred):
    smooth = 1e-6
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return 1 - ((2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth))

def combined_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    return bce + dice_loss(y_true, y_pred)

# Load U-Net and ResU-Net models with custom loss
custom_objects = {'combined_loss': combined_loss, 'dice_loss': dice_loss}
unet = load_model('Model Weights/fetal_head_segmentation_unet.h5', custom_objects=custom_objects)
resunet = load_model('Model Weights/fetal_head_segmentation_resunet.h5', custom_objects=custom_objects)

# Image Preprocessing
def preprocess_image(image):
    image = image / 255.0
    return resize(image, (512, 512), mode='constant', preserve_range=True)

# Postprocess the mask and fit an ellipse
def postprocess_mask(mask, original_shape):
    mask = resize(mask, original_shape, mode='constant', preserve_range=True)
    mask = mask > 0.5  # Convert to binary mask
    contours = measure.find_contours(mask, 0.5)
    
    if len(contours) == 0:
        return None  # No valid contours found

    contour = max(contours, key=len)  # Largest contour

    # Convert contour to OpenCV format (float32) and check if it has enough points
    contour = np.array(contour, dtype=np.float32)
    if len(contour) < 5:
        print("⚠️ Warning: Not enough points to fit an ellipse")
        return None  # Not enough points to fit an ellipse

    return cv2.fitEllipse(contour)


# Compute HC from the ellipse
def calculate_HC(ellipse, pixel_size):
    major_axis = ellipse[1][0]
    minor_axis = ellipse[1][1]
    HC = 1.62 * (major_axis + minor_axis)
    return HC * pixel_size  # Convert to mm

# Load dataset
df = pd.read_csv('Dataset/training_set_pixel_size_and_HC.csv')

# Track HC values
actual_HCs = []
unet_HCs = []
resunet_HCs = []

# Path to images
image_dir = 'Dataset/training_set/training_set/'

for _, row in df.iterrows():
    filename = row['filename'].strip()  # Remove spaces/newlines
    pixel_size = row['pixel size(mm)']
    actual_HC = row['head circumference (mm)']

    # Only consider images ending with "_HC.png"
    if not filename.endswith("_HC.png"):
        continue  # Skip non-HC images

    # Construct full image path
    image_path = os.path.join(image_dir, filename)
    if not os.path.exists(image_path):
        print(f"⚠️ Warning: Image {filename} not found at {image_path}")
        continue  # Skip this image

    # Load and preprocess image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"⚠️ Error: Could not read {filename}")
        continue

    preprocessed_image = preprocess_image(image)
    preprocessed_image = np.expand_dims(preprocessed_image, axis=(0, -1))  # Add batch & channel dims

    # Predict segmentation masks
    unet_mask = unet.predict(preprocessed_image)[0, :, :, 0]
    resunet_mask = resunet.predict(preprocessed_image)[0, :, :, 0]

    # Postprocess masks to get ellipses
    unet_ellipse = postprocess_mask(unet_mask, image.shape)
    resunet_ellipse = postprocess_mask(resunet_mask, image.shape)

    if unet_ellipse is None or resunet_ellipse is None:
        print(f"⚠️ Warning: No valid ellipse found for {filename}")
        continue

    # Calculate HC values
    unet_HC = calculate_HC(unet_ellipse, pixel_size)
    resunet_HC = calculate_HC(resunet_ellipse, pixel_size)

    actual_HCs.append(actual_HC)
    unet_HCs.append(unet_HC)
    resunet_HCs.append(resunet_HC)

# Compute MSE only if there are valid samples
if len(actual_HCs) > 0:
    unet_mse = mean_squared_error(actual_HCs, unet_HCs)
    resunet_mse = mean_squared_error(actual_HCs, resunet_HCs)
    print(f"✅ U-Net MSE: {unet_mse:.2f} mm²")
    print(f"✅ ResU-Net MSE: {resunet_mse:.2f} mm²")
else:
    print("❌ No valid samples to compute MSE.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 